In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*

In [ ]:
val model = Network("model")

// Create nodes
val classSwitch = ClassSwitch(model, "CS", arrayOf(intArrayOf(0, 1), intArrayOf(1, 0)))
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
val delay = Queue(model, "Delay", SchedStrategy.INF)

// Create job classes
val jobclass1 = ClosedClass(model, "Class1", 15, delay, 0)
val jobclass2 = ClosedClass(model, "Class2", 5, delay, 0)

In [ ]:
// Set service time distributions
// Queue1: Exponential for Class1, Erlang for Class2
queue1.setService(jobclass1, Exp.fitMean(1.5))  // mean = 1.5
queue1.setService(jobclass2, Erlang.fitMeanAndOrder(1.5, 2))  // mean = 1.5, order = 2

// Queue2: Erlang for Class1, Exponential for Class2
queue2.setService(jobclass1, Erlang.fitMeanAndOrder(1.5, 2))  // mean = 1.5, order = 2
queue2.setService(jobclass2, Exp.fitMean(1.5))  // mean = 1.5

// Delay: Exponential for both classes
delay.setService(jobclass1, Exp.fitMean(1.0))  // mean = 1
delay.setService(jobclass2, Exp.fitMean(1.0))  // mean = 1

In [ ]:
// Add network links
model.addLink(queue1, classSwitch)
model.addLink(queue2, classSwitch)
model.addLink(classSwitch, delay)
model.addLink(delay, queue1)
model.addLink(delay, queue2)

In [ ]:
// Set routing strategies
classSwitch.setRouting(jobclass1, RoutingStrategy.RAND)
queue1.setRouting(jobclass1, RoutingStrategy.RAND)
queue2.setRouting(jobclass1, RoutingStrategy.RAND)
delay.setRouting(jobclass1, RoutingStrategy.RROBIN)

classSwitch.setRouting(jobclass2, RoutingStrategy.RAND)
queue1.setRouting(jobclass2, RoutingStrategy.RAND)
queue2.setRouting(jobclass2, RoutingStrategy.RAND)
delay.setRouting(jobclass2, RoutingStrategy.WRROBIN, queue1, 1.0)
delay.setRouting(jobclass2, RoutingStrategy.WRROBIN, queue2, 2.0)

In [ ]:
// Solve with JMTval simoptions = JMT.defaultOptions()simoptions.verbose = truesimoptions.seed = 23000val solver = JMT(model, simoptions)val avgTable = solver.getAvgTable()avgTable.print()

This network demonstrates several advanced features:

1. **Multiple Job Classes**: Two classes with different populations (15 and 5 jobs)
2. **Class Switching**: Jobs can switch between classes using the ClassSwitch node
3. **Mixed Service Distributions**: 
   - Queue1: Exponential for Class1, Erlang-2 for Class2
   - Queue2: Erlang-2 for Class1, Exponential for Class2
4. **Different Routing Strategies**:
   - Random routing for Class1
   - Weighted round-robin for Class2 (1:2 ratio between Queue1 and Queue2)

The results show how the different service time distributions and routing strategies affect the performance metrics for each class.